メモ

Boson sampling の問題点
- Boson Samplingは確かにParm関数を効率的に計算可能。
- しかし、入力に光子数状態を使っている。例えば1光子状態を確率的に得る手法はこなれているが、決定論的に得るのはなかなか難しい。
- 確率的に得るとして、例えば各入力ポートに1光子状態がほしければ、各発生源が偶然同じタイミングで光子を吐き出すのを待つ必要がある。
- すると結局計算にかかる時間は指数関数的に増えて、古典に対する優位性なくない？

Gaussian Boson sampling による改善
- 入力は光子数状態でなく真空場で良い。代わりにスクイージングゲートを用意する。
- スクイージングが決定論的に行えれば良い。
- 全部Gaussian で処理できる(利点なのか...?)

In [ ]:
import strawberryfields as sf
from strawberryfields.ops import *
from strawberryfields.utils import random_interferometer

## Prepare Unitary Matrix

In [13]:
U = np.array([
 [ 0.219546940711-0.256534554457j, 0.611076853957+0.524178937791j,
    -0.102700187435+0.474478834685j,-0.027250232925+0.03729094623j],
 [ 0.451281863394+0.602582912475j, 0.456952590016+0.01230749109j,
    0.131625867435-0.450417744715j, 0.035283194078-0.053244267184j],
 [ 0.038710094355+0.492715562066j,-0.019212744068-0.321842852355j,
    -0.240776471286+0.524432833034j,-0.458388143039+0.329633367819j],
 [-0.156619083736+0.224568570065j, 0.109992223305-0.163750223027j,
    -0.421179844245+0.183644837982j, 0.818769184612+0.068015658737j]
])

# Networking and Run

In [14]:
#U = random_interferometer(4)
#print(U)

eng, q = sf.Engine(4) # 4 qumodes

with eng:
    # prepare the input squeezed states
    S = Sgate(1)
    All(S) | q

    # interferometer decompose unitary U to Rgate and BS
    Interferometer(U) | q
    
state = eng.run('gaussian') # Fock state is not used!

print("The network is following")
eng.print_applied()

#B = (np.dot(U, U.T) * np.tanh(1)) # 4 x 4 matrics

The network is following
Run 0:
Sgate(1, 0) | (q[0])
Sgate(1, 0) | (q[1])
Sgate(1, 0) | (q[2])
Sgate(1, 0) | (q[3])
Rgate(-3.124) | (q[0])
BSgate(0.9465, 0) | (q[0], q[1])
Rgate(2.724) | (q[2])
BSgate(0.09485, 0) | (q[2], q[3])
Rgate(-0.9705) | (q[1])
BSgate(0.7263, 0) | (q[1], q[2])
Rgate(-1.788) | (q[0])
BSgate(0.8246, 0) | (q[0], q[1])
Rgate(-0.9397) | (q[0])
Rgate(2.93) | (q[1])
Rgate(3.133) | (q[2])
Rgate(0.07904) | (q[3])
BSgate(-0.533, 0) | (q[2], q[3])
Rgate(2.45) | (q[2])
BSgate(-0.03962, 0) | (q[1], q[2])
Rgate(2.508) | (q[1])


In [15]:
measure_states = [[0,0,0,0], [1,1,0,0], [0,1,0,1], [1,1,1,1], [2,0,0,0]]
for i in measure_states:
    prob = state.fock_prob(i)
    print("|"+str(i[0])+str(i[1])+str(i[2])+str(i[3])+">", prob)

|0000> 0.17637844761413454
|1100> 0.0685595637122449
|0101> 0.0020560972589722857
|1111> 0.008342946399881914
|2000> 0.010312945253440235


all_fock_probs() is not supported by Gaussian states!

It's because computing the Fock probabilities of states in the Gaussian representation has exponential scaling - while this is fine for computing particular Fock basis probabilities, it becomes computationally demanding to return all Fock state probabilities using the Gaussian backend.

Gaussian Stateを光子数基底へ射影する(=光子数状態の確率を求める)のは計算負荷が高い！

# Classical, analytical solution

In [16]:
from itertools import permutations
from scipy.special import factorial

def Haf(M):
    n=len(M)
    m=int(n/2)
    haf=0.0
    for i in permutations(range(n)):
        prod=1.0
        for j in range(m):
            prod*=M[i[2*j],i[2*j+1]]
        haf+=prod
    return haf/(factorial(m)*(2**m))